# Libraries

In [1]:
!pip install Sastrawi
!pip install clean-text
!pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 5.4 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171034 sha256=32df5d5d677cecab031a826290bdc424062e7ef6712c93c3e15320c51a1d1f4f
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from cleantext import clean
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
import sklearn

In [ ]:
# !pip freeze > requirements.txt

# Get Data from ES and prepare DF

In [4]:
import requests
import json

In [5]:
# Importing Data
url = 'https://raw.githubusercontent.com/TaillessTanuki/Sosmed-twt_Banggai_Sent/main/vadered_text1_edited.csv'

# Create a DataFrame
df = pd.read_csv(url,encoding='utf-8')
print(df.head())

                                                text sentiment
0  15 Menit sebelm tutup kantor minta ditunggu bi...   negatif
1  Salah satu hal teraneh di dunia ini versiku ad...    netral
2  W benci bgt kalo co cod tapi alamatnya salah h...   negatif
3  Dear @Erdioo Radio *RADIO GALASIKA FM LUWUK (@...   negatif
4                    Kenapa batui itu dinginnya aneh   negatif


In [11]:
len(df)

562

In [7]:
# Drop Unnamed column
df.drop(df.columns[df.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)

In [10]:
df.head()

,text,sentiment
0,15 Menit sebelm tutup kantor minta ditunggu bi...,negatif
1,Salah satu hal teraneh di dunia ini versiku ad...,netral
2,W benci bgt kalo co cod tapi alamatnya salah h...,negatif
3,Dear @Erdioo Radio *RADIO GALASIKA FM LUWUK (@...,negatif
4,Kenapa batui itu dinginnya aneh,negatif


# Preprocessing

## Delete Duplicates

In [8]:
df.duplicated(subset='text').sum()


0

In [ ]:
df = df.drop_duplicates(subset='title')

## Deleting Blank (Nan) Rows

In [9]:
df = df[df['text'] != '']

In [ ]:
#row_index = 127  # Index of the row you want to print (Python indexing starts from 0)
#columns_to_print = ['title', 'text']  # List of column names to display

# Use .loc[] to select the row by its index and specify the columns to display
#row_data = df.loc[row_index, columns_to_print]

#print(row_data)

## Case Folding

In [ ]:
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

In [ ]:
# Buat kolom tambahan untuk data ulasan yang telah dicasefolding
df['lwr'] = df['text'].apply(clean_lower)
df['lwr']

0      kabar luwuk –  suasana pasar murah bersubsidi ...
1      kabar luwuk  –  film horor“piara mayat”,kreati...
2      kabar luwuk  –festival malabot tumbe 2023kemba...
3      “kerja cepat dan terkoordinir dalam penerbitan...
4      kabar luwuk  –bawaslu banggaiperkuat koordinas...
                             ...                        
574    banggai raya-satu gudang bahan aluminium di ja...
575    laporan wartawan tribunpalu.com, asnawi zikri ...
576    banggai raya-lebih 500 pebulutangkis dari kabu...
577    laporan wartawan tribunpalu.com, asnawi zikri ...
578    laporan wartawan tribunpalu.com, asnawi zikri ...
Name: lwr, Length: 579, dtype: object

In [ ]:
df.head()

,final_review,title,text,lwr
0,positif,Mobil Sim Keliling Polres Banggai Meriahkan Pa...,KABAR LUWUK – Suasana Pasar Murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...
1,positif,"Film Horor “Piara Mayat”, Kreativitas Anak Mu...","KABAR LUWUK – Film Horor“Piara Mayat”,Kreati...","kabar luwuk – film horor“piara mayat”,kreati..."
2,positif,Festival Malabot Tumbe 2023 Kembali Meriahkan ...,KABAR LUWUK –Festival Malabot Tumbe 2023Kemba...,kabar luwuk –festival malabot tumbe 2023kemba...
3,positif,"Rakor Kampanye Damai Pemilu 2024, Polres Bangg...",“Kerja Cepat dan Terkoordinir dalam Penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...
4,positif,Bawaslu Banggai Perkuat Koordinasi dengan Panw...,KABAR LUWUK –Bawaslu BanggaiPerkuat Koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...


In [ ]:
print(df['text'][30])
print("_______________________________________________________________________________________________________________")
print(df['lwr'][30])

KABAR LUWUK  –Dinas Sosial BanggaiSegera Salurkan 2.42 MiliarProgram Gercep Gaskan Berdaya2023.Dinas Sosial Kabupaten Banggaimenerima bantuan sebesar 2.42 miliar rupiah dalam program Gerak Cepat Pengentasan Kemiskinan Berbasis Pemberdayaan Masyarakat (GERCEP GASKAN BERDAYA).Bantuan ini ditujukan untuk 242 keluarga penerima manfaat di lima kecamatan, memberikan harapan bagi warga di Kecamatan Luwuk (35 KPM), Luwuk Selatan (28 KPM), Pagimana (95 KPM), Bunta (47 KPM), dan Nuhon (37 KPM). Selasa 28/11/2023.Kepala Dinas Sosial Kabupaten Banggai, Irpan Poma melalui Kepala Bidang Fakir Miskis,Ronald Putje,menyampaikan kepada media bahwa dana bantuan ini merupakan sumbangan langsung dari Pemerintah Provinsi Sulawesi Tengah.Dinas Sosial Kabupaten Banggai bertanggung jawab dalam memfasilitasi distribusi bantuan dan ikut serta dalam pengawasan ketat agar dana tersebut benar-benar sampai kepada penerima manfaat yang tepat.Data yang tersedia menunjukkan bahwa 242 Keluarga Penerima Manfaat yang terd

## Cleansing

### Remove Numbers

In [ ]:
def remove_numbers(text):
  text = re.sub('[0-9]+', '', text)
  return text

In [ ]:
df['clean_number'] = df['lwr'].apply(remove_numbers)
df['clean_number']

0      kabar luwuk –  suasana pasar murah bersubsidi ...
1      kabar luwuk  –  film horor“piara mayat”,kreati...
2      kabar luwuk  –festival malabot tumbe kembali m...
3      “kerja cepat dan terkoordinir dalam penerbitan...
4      kabar luwuk  –bawaslu banggaiperkuat koordinas...
                             ...                        
574    banggai raya-satu gudang bahan aluminium di ja...
575    laporan wartawan tribunpalu.com, asnawi zikri ...
576    banggai raya-lebih  pebulutangkis dari kabupat...
577    laporan wartawan tribunpalu.com, asnawi zikri ...
578    laporan wartawan tribunpalu.com, asnawi zikri ...
Name: clean_number, Length: 579, dtype: object

In [ ]:
df.tail()

,final_review,title,text,lwr,clean_number
574,negatif,Gudang Alumunium Kompleks Pertokoan Luwuk Terb...,BANGGAI RAYA-Satu gudang bahan aluminium di Ja...,banggai raya-satu gudang bahan aluminium di ja...,banggai raya-satu gudang bahan aluminium di ja...
575,positif,583 Peserta Ramaikan Turnamen Badminton di Ban...,"Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ..."
576,negatif,Lebih 500 Pebulutangkis Berlaga di Open Turnam...,BANGGAI RAYA-Lebih 500 pebulutangkis dari kabu...,banggai raya-lebih 500 pebulutangkis dari kabu...,banggai raya-lebih pebulutangkis dari kabupat...
577,negatif,"Demi Penuhi Kehidupan Sehari-hari, Pria di Ban...","Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ..."
578,negatif,Gudang Aluminium di Pertokoan Luwuk Banggai Te...,"Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ..."


### Remove Punctuation

In [ ]:
#Remove Puncutuation
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text

In [ ]:
# Buat kolom tambahan untuk data description yang telah diremovepunctuation
df['clean_punct'] = df['clean_number'].apply(clean_punct)
df['clean_punct']

0      kabar luwuk    suasana pasar murah bersubsidi ...
1      kabar luwuk     film horor piara mayat kreativ...
2      kabar luwuk   festival malabot tumbe kembali m...
3       kerja cepat dan terkoordinir dalam penerbitan...
4      kabar luwuk   bawaslu banggaiperkuat koordinas...
                             ...                        
574    banggai raya satu gudang bahan aluminium di ja...
575    laporan wartawan tribunpalu com asnawi zikri t...
576    banggai raya lebih  pebulutangkis dari kabupat...
577    laporan wartawan tribunpalu com asnawi zikri t...
578    laporan wartawan tribunpalu com asnawi zikri t...
Name: clean_punct, Length: 579, dtype: object

In [ ]:
df.head()


,final_review,title,text,lwr,clean_number,clean_punct
0,positif,Mobil Sim Keliling Polres Banggai Meriahkan Pa...,KABAR LUWUK – Suasana Pasar Murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi ...
1,positif,"Film Horor “Piara Mayat”, Kreativitas Anak Mu...","KABAR LUWUK – Film Horor“Piara Mayat”,Kreati...","kabar luwuk – film horor“piara mayat”,kreati...","kabar luwuk – film horor“piara mayat”,kreati...",kabar luwuk film horor piara mayat kreativ...
2,positif,Festival Malabot Tumbe 2023 Kembali Meriahkan ...,KABAR LUWUK –Festival Malabot Tumbe 2023Kemba...,kabar luwuk –festival malabot tumbe 2023kemba...,kabar luwuk –festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali m...
3,positif,"Rakor Kampanye Damai Pemilu 2024, Polres Bangg...",“Kerja Cepat dan Terkoordinir dalam Penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan...
4,positif,Bawaslu Banggai Perkuat Koordinasi dengan Panw...,KABAR LUWUK –Bawaslu BanggaiPerkuat Koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinas...


### Delete whitespaces

In [ ]:
def normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

In [ ]:
df['clean_double_ws'] = df['clean_punct'].apply(normalize_whitespace)
df['clean_double_ws']

0      kabar luwuk suasana pasar murah bersubsidi di ...
1      kabar luwuk film horor piara mayat kreativitas...
2      kabar luwuk festival malabot tumbe kembali mer...
3      kerja cepat dan terkoordinir dalam penerbitan ...
4      kabar luwuk bawaslu banggaiperkuat koordinasi ...
                             ...                        
574    banggai raya satu gudang bahan aluminium di ja...
575    laporan wartawan tribunpalu com asnawi zikri t...
576    banggai raya lebih pebulutangkis dari kabupate...
577    laporan wartawan tribunpalu com asnawi zikri t...
578    laporan wartawan tribunpalu com asnawi zikri t...
Name: clean_double_ws, Length: 579, dtype: object

In [ ]:
df.tail()

,final_review,title,text,lwr,clean_number,clean_punct,clean_double_ws
574,negatif,Gudang Alumunium Kompleks Pertokoan Luwuk Terb...,BANGGAI RAYA-Satu gudang bahan aluminium di Ja...,banggai raya-satu gudang bahan aluminium di ja...,banggai raya-satu gudang bahan aluminium di ja...,banggai raya satu gudang bahan aluminium di ja...,banggai raya satu gudang bahan aluminium di ja...
575,positif,583 Peserta Ramaikan Turnamen Badminton di Ban...,"Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...",laporan wartawan tribunpalu com asnawi zikri t...,laporan wartawan tribunpalu com asnawi zikri t...
576,negatif,Lebih 500 Pebulutangkis Berlaga di Open Turnam...,BANGGAI RAYA-Lebih 500 pebulutangkis dari kabu...,banggai raya-lebih 500 pebulutangkis dari kabu...,banggai raya-lebih pebulutangkis dari kabupat...,banggai raya lebih pebulutangkis dari kabupat...,banggai raya lebih pebulutangkis dari kabupate...
577,negatif,"Demi Penuhi Kehidupan Sehari-hari, Pria di Ban...","Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...",laporan wartawan tribunpalu com asnawi zikri t...,laporan wartawan tribunpalu com asnawi zikri t...
578,negatif,Gudang Aluminium di Pertokoan Luwuk Banggai Te...,"Laporan Wartawan TribunPalu.com, Asnawi Zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...","laporan wartawan tribunpalu.com, asnawi zikri ...",laporan wartawan tribunpalu com asnawi zikri t...,laporan wartawan tribunpalu com asnawi zikri t...


## Stopword Removal

### Get custom stopwords

In [ ]:
url1 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-stopwords_banggai_addedwords.csv'
df_stopword1 = pd.read_csv(url1, delimiter = ";",encoding='cp1252')

In [ ]:
url2 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-Stopwords(MasDevid).csv'
df_stopword2 = pd.read_csv(url2, delimiter = ";",encoding='cp1252')

In [ ]:
# df_stopword.info()

In [ ]:
# df_stopword

In [ ]:
df.head()

In [ ]:
custom_stopword1 = list(df_stopword1['stopword'])
custom_stopword2 = list(df_stopword2['Stopwords'])

### Removal

In [ ]:
# Prepare stopword remover
#factory = StopWordRemoverFactory()
#stopwords = factory.get_stop_words() + custom_stopword
#print(stopwords)

In [ ]:
# using only sastrawy
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words() + custom_stopword1 + custom_stopword2

In [ ]:
len(stopwords)

946

In [ ]:
dictionary = ArrayDictionary(stopwords)
stopword = StopWordRemover(dictionary)


In [ ]:
# stopwords

In [ ]:
def filtering(text):
  stop = stopword.remove(text)
  return stop

In [ ]:
df.head()

,final_review,title,text,lwr,clean_number,clean_punct,clean_double_ws
0,positif,Mobil Sim Keliling Polres Banggai Meriahkan Pa...,KABAR LUWUK – Suasana Pasar Murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi di ...
1,positif,"Film Horor “Piara Mayat”, Kreativitas Anak Mu...","KABAR LUWUK – Film Horor“Piara Mayat”,Kreati...","kabar luwuk – film horor“piara mayat”,kreati...","kabar luwuk – film horor“piara mayat”,kreati...",kabar luwuk film horor piara mayat kreativ...,kabar luwuk film horor piara mayat kreativitas...
2,positif,Festival Malabot Tumbe 2023 Kembali Meriahkan ...,KABAR LUWUK –Festival Malabot Tumbe 2023Kemba...,kabar luwuk –festival malabot tumbe 2023kemba...,kabar luwuk –festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali mer...
3,positif,"Rakor Kampanye Damai Pemilu 2024, Polres Bangg...",“Kerja Cepat dan Terkoordinir dalam Penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan ...
4,positif,Bawaslu Banggai Perkuat Koordinasi dengan Panw...,KABAR LUWUK –Bawaslu BanggaiPerkuat Koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinasi ...


In [ ]:
#df.drop(['filtered'], axis=1, inplace=True)

In [ ]:
df['filtered'] = df['clean_double_ws'].apply(filtering)
df['filtered']

0      luwuk suasana pasar murah bersubsidi semarak k...
1      luwuk film horor piara mayat kreativitasanak m...
2      luwuk festival malabot tumbe meriahkankabupate...
3      kerja cepat terkoordinir penerbitan sttp luwuk...
4      luwuk bawaslu banggaiperkuat koordinasi dengan...
                             ...                        
574    raya gudang bahan aluminium jalan teuku umar k...
575    wartawan asnawi tribunpalu com banggai membuka...
576    raya pebulutangkis kabupatenkota sulawesi berl...
577    wartawan asnawi tribunpalu com banggai pria ne...
578    wartawan asnawi tribunpalu com banggai gudang ...
Name: filtered, Length: 579, dtype: object

In [ ]:
len(df)

579

In [ ]:
print(df['clean_double_ws'][55])
print(df['filtered'][55])

## Stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

In [ ]:
#factory = StemmerFactory()
#stemmer = factory.create_stemmer()

In [ ]:
def stem(text):
  return stemmer.stem(text)

In [ ]:
df.head()

In [ ]:
df['stemmed'] = df['filtered'].apply(stem)
df['stemmed']

0      luwuk suasana pasar murah subsidi semarak hadi...
1      luwuk film horor piara mayat kreativitasanak m...
2      luwuk festival malabot tumbe meriahkankabupate...
3      kerja cepat terkoordinir terbit sttp luwuk rak...
4      luwuk bawaslu banggaiperkuat koordinasi dengan...
                             ...                        
574    raya gudang bahan aluminium jalan teuku umar k...
575    wartawan asnawi tribunpalu com bangga buka aca...
576    raya pebulutangkis kabupatenkota sulawesi laga...
577    wartawan asnawi tribunpalu com bangga pria nek...
578    wartawan asnawi tribunpalu com bangga gudang b...
Name: stemmed, Length: 579, dtype: object

In [ ]:
df.head()

## Export df to csv (to cutting time if we want to stepback process)

In [ ]:
df.to_csv("stemmed_words_vader_sentiment1.csv", index=False)

## Importing df rom csv (cutting time if we want to stepback process)

In [ ]:
# to skip stemming process
stemmed_url= 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/stemmed_words2_tanpa_token_stopwordk2.csv'

df = pd.read_csv(stemmed_url,encoding='utf-8')

df = df.astype(str)

In [ ]:
df.head()


In [ ]:
len(df)

149

In [ ]:
print(df['filtered'][55])
print(df['stemmed'][55])

tim inafis polres banggaiselidiki pencurian desa bubung kerugian rp juta tim indonesianautomatic fingerprints identification system inafissatreskrimpolres banggaitengah intensif penyelidikan terkait pencurian terjadi didesa bubungkecamatan luwuk selatan kejadian diketahui kamis wita dilaporkan nomor polisi lpbxiispktres banggaipolda sulteng kamis desember keterangankasat reskrim polres akp tondytim inafis olah kejadian perkara tkp mengumpulkan bukti petunjuk terkait kejadian korban nama arifin morintoh melaporkan kejadian pada pukul wita menemukan rumahnya telah dirusak pencuri saksi saksi terlibat kejadian seperti indah permata sari morintoh keterangan saat rumahnya di desa bubung menemukan rumah berantakan ditemukannya jendela rumah samping dirusak cara dicungkil akp anjas suyono penjaga rumah menjelaskan pada rabu pukul wita meninggalkan rumah dalam keadaan terkunci ia terkejut indah memberitahu rumah telah sasaran pencurian akibat kejadian hilang unit keyboard merk yamaha kg kemiri

In [ ]:
print(len(df['filtered'][796]))
print(len(df['stemmed'][796]))

1167
990


## Tokenizing

In [ ]:
def tokenize(text):
  tokens = nltk.tokenize.word_tokenize(text)
  return tokens

In [ ]:
df.head()

In [ ]:
df['tokenized'] = df['stemmed'].apply(tokenize)
df['tokenized']

0      [luwuk, suasana, pasar, murah, subsidi, semara...
1      [luwuk, film, horor, piara, mayat, kreativitas...
2      [luwuk, festival, malabot, tumbe, meriahkankab...
3      [kerja, cepat, terkoordinir, terbit, sttp, luw...
4      [luwuk, bawaslu, banggaiperkuat, koordinasi, d...
                             ...                        
574    [raya, gudang, bahan, aluminium, jalan, teuku,...
575    [wartawan, asnawi, tribunpalu, com, bangga, bu...
576    [raya, pebulutangkis, kabupatenkota, sulawesi,...
577    [wartawan, asnawi, tribunpalu, com, bangga, pr...
578    [wartawan, asnawi, tribunpalu, com, bangga, gu...
Name: tokenized, Length: 579, dtype: object

In [ ]:
print(df['filtered'][0])
print(df['tokenized'][0])

warga penjaringan utara bernama josua yakin pembangunan infrastruktur berjalan status kota negara dilihat infrastruktur berjalan pembangunannya berubah pembangunan tiba berhenti dia dihubungi kompas com jumat josua pembangunan infrastruktur berhenti lantaran pusat bisnis perekonomian baca status kota dicabut warga konser ikn semoga tiket pesawatnya murah kata pemerintah serta merta memfokuskan perhatian kota nusantara ikn menuturkan progres pembangunan infrastruktur bakal dibuat lambat cenderung cepat sih saya enggak akan fokus ikn doang pembangunan infrastruktur bakal tetap berjalan cukup pesat josua serupa dituturkan warga pademangan utara bernama anggita nantinya pembangunan infrastruktur lambat berarti fokus pemerintah ikn terkendala sisi kayak cuaca dan ekonomi anggita dihubungi jumat jika faktor ekonomi berkaitan pembangunan ikn anggita menyayangkannya baca pendapat warga soal hilangnya status daerah khusus kota ada sedih dan cuek jika faktor ekonomi proyek infrastruktur yang sed

In [ ]:
print(len(df['stemmed'][0]))
print(len(df['tokenized'][0]))

1584
272


##Second Stopwords

In [ ]:
df.head()

In [ ]:
# Define a function to remove stopwords from a list of tokens
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords]

# Apply the remove_stopwords function to each row in the 'tokenized' column
df['filtered2'] = df['tokenized'].apply(lambda tokens: remove_stopwords(tokens))

# Display the updated DataFrame
print(df.head())

In [ ]:
#df.drop(columns=['tokenized'], inplace=True)

In [ ]:
len(stopwords)

946

In [ ]:
df.head()

In [ ]:
print(df['tokenized'][55])
print(df['filtered2'][55])

['tim', 'inafis', 'polres', 'banggaiselidiki', 'curi', 'desa', 'bubung', 'rugi', 'rp', 'juta', 'tim', 'indonesianautomatic', 'fingerprints', 'identification', 'system', 'inafissatreskrimpolres', 'banggaitengah', 'intensif', 'lidi', 'kait', 'curi', 'jadi', 'desa', 'bubungkecamatan', 'luwuk', 'selatan', 'jadi', 'tahu', 'kamis', 'wita', 'lapor', 'nomor', 'polisi', 'lpbxiispktres', 'banggaipolda', 'sulteng', 'kamis', 'desember', 'keterangankasat', 'reskrim', 'polres', 'akp', 'tondytim', 'inafis', 'olah', 'jadi', 'perkara', 'tkp', 'kumpul', 'bukti', 'tunjuk', 'kait', 'jadi', 'korban', 'nama', 'arifin', 'morintoh', 'lapor', 'jadi', 'pada', 'pukul', 'wita', 'temu', 'rumah', 'telah', 'rusak', 'curi', 'saksi', 'saksi', 'libat', 'jadi', 'seperti', 'indah', 'permata', 'sari', 'morintoh', 'terang', 'saat', 'rumah', 'di', 'desa', 'bubung', 'temu', 'rumah', 'beranta', 'temu', 'jendela', 'rumah', 'samping', 'rusak', 'cara', 'cungkil', 'akp', 'anjas', 'suyono', 'jaga', 'rumah', 'jelas', 'pada', 'rabu'

# Text Sentiment Classification

In [ ]:
# For manual Review
#import random

# Set the number of random samples you want to select
#num_samples = 100  # Adjust this number as needed

# Select random samples from the DataFrame
#random_indices = random.sample(range(len(df)), num_samples)
#random_data = df.iloc[random_indices][["title", "text"]]

# Save random data to a CSV file
#random_data.to_csv("random_data.csv")


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562 entries, 0 to 561
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       562 non-null    object
 1   sentiment  562 non-null    object
dtypes: object(2)
memory usage: 8.9+ KB


## Preprocessing (Text Classification)

In [ ]:
df.head()

,text,sentiment
0,15 Menit sebelm tutup kantor minta ditunggu bi...,negatif
1,Salah satu hal teraneh di dunia ini versiku ad...,netral
2,W benci bgt kalo co cod tapi alamatnya salah h...,negatif
3,Dear @Erdioo Radio *RADIO GALASIKA FM LUWUK (@...,negatif
4,Kenapa batui itu dinginnya aneh,negatif


In [34]:
# Spliting database to train_set and test_set
from sklearn.model_selection import train_test_split

X = df['text']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42)

In [35]:
X_train.head()

464    Capek ngga motil mks-jogja-solo-jogja-sby-mks-...
0      15 Menit sebelm tutup kantor minta ditunggu bi...
11       Ketika hati so dirumah tapi raga masih di batui
448                        rindu vibes pildun di luwuk e
22                              Pindah kampung otw batui
Name: text, dtype: object

In [36]:
print(y_test.unique())


#df['Topic'].nunique()

['netral' 'positif' 'negatif']


In [37]:
# CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()


X_train_counts = count_vect.fit_transform(X_train)


In [ ]:
X_train_counts

<438x2657 sparse matrix of type '<class 'numpy.int64'>'
	with 5826 stored elements in Compressed Sparse Row format>

In [ ]:
X_train.shape

(438,)

In [ ]:
X_train_counts.shape

(438, 2657)

In [39]:
# TFIDFVectorizer and LinearSVC with pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
#from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.ensemble import RandomForestClassifier


from sklearn.pipeline import Pipeline

sent_clf_twt = Pipeline([('tfidf', TfidfVectorizer()),('clf',LinearSVC())])
sent_clf_twt.fit(X_train, y_train)



Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [40]:
predictions = sent_clf_twt.predict(X_test)

In [41]:
# Checking performance using confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))
print(f"Accuracy {metrics.accuracy_score(y_test, predictions)}")

[[10 15  3]
 [ 3 51  6]
 [ 2 18  5]]
              precision    recall  f1-score   support

     negatif       0.67      0.36      0.47        28
      netral       0.61      0.85      0.71        60
     positif       0.36      0.20      0.26        25

    accuracy                           0.58       113
   macro avg       0.54      0.47      0.48       113
weighted avg       0.57      0.58      0.55       113

Accuracy 0.584070796460177


# Input Data

In [ ]:
elasticsearch_url = 'https://es-1.datanusantara.com:9201'
username = 'adminkibana'
password = 'kibanaPASS!@#2019'
index_endpoint = 'socmed-twitter'

In [ ]:
query = {
  "_source": ["target", "source","text"],
  "size": 176,
  "query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "match_all": {}
        },
        {
          "bool": {
            "should": [


              {
                "match_phrase": {
                  "target": "banggai"
                }
              }
            ],
            "minimum_should_match": 1


          }
        }
      ]
    }
  }
}
search_url = f"{elasticsearch_url}/{index_endpoint}/_search"
response = requests.get(search_url, json=query, auth=(username, password))


In [ ]:
search_results = response.json()
# print(json.dumps(search_results, indent=2))
print(len(search_results['hits']['hits']))

176


In [ ]:
# Input
# Parse the response to extract relevant data
search_results = response.json()

# Process the data and create a DataFrame
data = []
for hit in search_results.get("hits", {}).get("hits", []):
    target = hit["_source"]["target"]
    source = hit["_source"]["source"]
    text = hit["_source"]["text"]

    data.append({"target": target,  "source": source, "text": text})

# Create a DataFrame
input_df = pd.DataFrame(data)
print(input_df.head())

    target    source                                               text
0  banggai    #luwuk         Ada bocoran Nurut Apa Kata Mama Season 2?!
1  banggai    #luwuk  【NEW】PRO X WIRELESS KEYBOARD\n\nプロ選手にも一般ゲーマーにも...
2  banggai  #banggai  We're thrilled to announce that HCLTech has be...
3  banggai  #banggai  Elevate your enterprise reliability and busine...
4  banggai  #banggai         Ada bocoran Nurut Apa Kata Mama Season 2?!


In [ ]:
input_df.head()

,target,source,text
0,banggai,#luwuk,Ada bocoran Nurut Apa Kata Mama Season 2?!
1,banggai,#luwuk,【NEW】PRO X WIRELESS KEYBOARD\n\nプロ選手にも一般ゲーマーにも...
2,banggai,#banggai,We're thrilled to announce that HCLTech has be...
3,banggai,#banggai,Elevate your enterprise reliability and busine...
4,banggai,#banggai,Ada bocoran Nurut Apa Kata Mama Season 2?!


## Preprocessing

In [ ]:
# Delete Duplicates
input_df.duplicated(subset='title').sum()
input_df = input_df.drop_duplicates(subset='title')

# Delete Blank text (Nan) Rows
input_df = input_df[input_df['text'] != '']

# Case Folding
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

input_df['lwr'] = input_df['text'].apply(clean_lower)
input_df['lwr']


# Remove Numbers
def remove_numbers(text):
  text = re.sub('[0-9]+', '', text)
  return text

input_df['clean_number'] = input_df['lwr'].apply(remove_numbers)
input_df['clean_number']


# Remove Punctuation
clean_spcl = re.compile('[/(){}\[\]\|@,;]')
clean_symbol = re.compile('[^0-9a-z]')
def clean_punct(text):
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    return text

input_df['clean_punct'] = input_df['clean_number'].apply(clean_punct)
input_df['clean_punct']


# Delete Whitespaces
def normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

input_df['clean_double_ws'] = input_df['clean_punct'].apply(normalize_whitespace)
input_df['clean_double_ws']


0      kabar luwuk suasana pasar murah bersubsidi di ...
1      kabar luwuk film horor piara mayat kreativitas...
2      kabar luwuk festival malabot tumbe kembali mer...
3      kerja cepat dan terkoordinir dalam penerbitan ...
4      kabar luwuk bawaslu banggaiperkuat koordinasi ...
                             ...                        
995    terasbanggai com bawaslu banggai akan meningka...
996    banggai raya bupati banggai ir h amirudin mm a...
997    terasbanggai com setelah sempat tertunda masa ...
998    banggai januari bertempat di ruang sidang pada...
999    banggai raya bawaslu kabupaten banggai menggel...
Name: clean_double_ws, Length: 966, dtype: object

In [ ]:
len(input_df)

966

In [ ]:
input_df.head()

In [ ]:
# Adding Stopwords
url1 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-stopwords_banggai_addedwords.csv'
df_stopword1 = pd.read_csv(url1, delimiter = ";",encoding='cp1252')

url2 = 'https://raw.githubusercontent.com/TaillessTanuki/Berita_Banggai/main/ID-Stopwords(MasDevid).csv'
df_stopword2 = pd.read_csv(url2, delimiter = ";",encoding='cp1252')

custom_stopword1 = list(df_stopword1['stopword'])
custom_stopword2 = list(df_stopword2['Stopwords'])

factory = StopWordRemoverFactory()
stopwords = factory.get_stop_words() + custom_stopword1 + custom_stopword2


# 1st Stopwords Removal
def filtering(text):
  stop = stopword.remove(text)
  return stop

input_df['filtered'] = input_df['clean_double_ws'].apply(filtering)
input_df['filtered']


# Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

def stem(text):
  return stemmer.stem(text)

input_df['stemmed'] = input_df['filtered'].apply(stem)
input_df['stemmed']


# Tokenizing
def tokenize(text):
  tokens = nltk.tokenize.word_tokenize(text)
  return tokens

input_df['tokenized'] = input_df['stemmed'].apply(tokenize)
input_df['tokenized']


# 2nd Stopwords Removal
# Define a function to remove stopwords from a list of tokens
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopwords]

# Apply the remove_stopwords function to each row in the 'tokenized' column
input_df['filtered2'] = input_df['tokenized'].apply(lambda tokens: remove_stopwords(tokens))

In [ ]:
input_df.head()

,target,title,text,lwr,clean_number,clean_punct,clean_double_ws,filtered,stemmed,tokenized,filtered2,text2
0,banggai,Mobil Sim Keliling Polres Banggai Meriahkan Pa...,KABAR LUWUK – Suasana Pasar Murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk – suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi ...,kabar luwuk suasana pasar murah bersubsidi di ...,luwuk suasana pasar murah bersubsidi semarak k...,luwuk suasana pasar murah subsidi semarak hadi...,"[luwuk, suasana, pasar, murah, subsidi, semara...","[suasana, pasar, murah, subsidi, semarak, hadi...",suasana pasar murah subsidi semarak hadir lalu...
1,banggai,"Film Horor “Piara Mayat”, Kreativitas Anak Mu...","KABAR LUWUK – Film Horor“Piara Mayat”,Kreati...","kabar luwuk – film horor“piara mayat”,kreati...","kabar luwuk – film horor“piara mayat”,kreati...",kabar luwuk film horor piara mayat kreativ...,kabar luwuk film horor piara mayat kreativitas...,luwuk film horor piara mayat kreativitasanak m...,luwuk film horor piara mayat kreativitasanak m...,"[luwuk, film, horor, piara, mayat, kreativitas...","[film, horor, piara, mayat, kreativitasanak, m...",film horor piara mayat kreativitasanak muda ba...
2,banggai,Festival Malabot Tumbe 2023 Kembali Meriahkan ...,KABAR LUWUK –Festival Malabot Tumbe 2023Kemba...,kabar luwuk –festival malabot tumbe 2023kemba...,kabar luwuk –festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali m...,kabar luwuk festival malabot tumbe kembali mer...,luwuk festival malabot tumbe meriahkankabupate...,luwuk festival malabot tumbe meriahkankabupate...,"[luwuk, festival, malabot, tumbe, meriahkankab...","[festival, malabot, tumbe, meriahkankabupaten,...",festival malabot tumbe meriahkankabupaten laut...
3,banggai,"Rakor Kampanye Damai Pemilu 2024, Polres Bangg...",“Kerja Cepat dan Terkoordinir dalam Penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,“kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan...,kerja cepat dan terkoordinir dalam penerbitan ...,kerja cepat terkoordinir penerbitan sttp luwuk...,kerja cepat terkoordinir terbit sttp luwuk rak...,"[kerja, cepat, terkoordinir, terbit, sttp, luw...","[kerja, cepat, terkoordinir, terbit, sttp, rak...",kerja cepat terkoordinir terbit sttp rakor kam...
4,banggai,Bawaslu Banggai Perkuat Koordinasi dengan Panw...,KABAR LUWUK –Bawaslu BanggaiPerkuat Koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk –bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinas...,kabar luwuk bawaslu banggaiperkuat koordinasi ...,luwuk bawaslu banggaiperkuat koordinasi dengan...,luwuk bawaslu banggaiperkuat koordinasi dengan...,"[luwuk, bawaslu, banggaiperkuat, koordinasi, d...","[bawaslu, banggaiperkuat, koordinasi, denganpa...",bawaslu banggaiperkuat koordinasi denganpanwas...


In [ ]:
# input_df.drop(columns=['text2'], inplace=True)

In [ ]:
# Convert tokenized text to string representation
#text_strings = [' '.join(doc) for doc in df['tokenized']]
input_df['text2'] = input_df['filtered2'].apply(lambda x: ' '.join(x))


In [ ]:
input_df.head()

## Convert input_df to csv

In [ ]:
input_df.to_csv("preprocesed_input_df.csv", index=False)

# Predicting Input Data

In [ ]:

# Assuming 'selected_features' is a list of feature names used during model training
X_input = input_df['text2']

# 3. Predict using the model
predictions = text_sent_clf.predict(X_input)

# 4. Add predictions to a new column in the DataFrame
input_df['final_review'] = predictions

In [ ]:
input_df.head()

## Convert input_df to csv and excel

In [ ]:
# Assuming 'df' is your existing DataFrame and 'columns_to_keep' is a list of column names you want to keep
columns_to_keep = ['title', 'text', 'final_review']  # Adjust with your desired column names

# Create a new DataFrame with only the selected columns
final_text_sent_cls1 = input_df[columns_to_keep].copy()

final_text_sent_cls1.to_csv('final_text_sent_cls1.csv', index=False)
final_text_sent_cls1.to_excel('final_text_sent_cls1.xlsx', index=False)

## Pickling Model

In [42]:
import pickle
#pickle.dump(model, open('model_name_you_want_to_save_as.sav', 'wb'))
pickle.dump(sent_clf_twt, open('sentiment_classifier_twt1.0.sav', 'wb'))